In [ ]:
import logging

import ssl4rs.utils

logger = ssl4rs.utils.logging.setup_logging_for_analysis_script(logging.INFO)
data_root_path = ssl4rs.utils.config.get_data_root_dir()
data_path = data_root_path / "fmow" / "fmow-rgb" / ".deeplake"
logger.info(f"Will attempt to load dataset at: {data_path}")

In [ ]:
import ssl4rs.data.parsers.fmow

instance_parser = ssl4rs.data.parsers.fmow.DeepLakeParser(
    dataset_path_or_object=data_path,
    parsing_strategy="instances",  # each 'sample' will be a unique instance with multiple images
)
instance_parser.summary()

In [ ]:
# we display multiple images loaded in parallel for a single instance at a time
import matplotlib.pyplot as plt
import numpy as np
import PIL.ImageDraw
import torchvision.transforms.functional

max_display_count = 5

display_instance_idx = np.random.randint(len(instance_parser))
print(f"Will display the different images of instance#{display_instance_idx}...")
batch = instance_parser[display_instance_idx]
images = batch["image"]  # list of rgb images, already decoded by default
bboxes = batch["bbox"]  # list of LTRB, pixel-wise instance bounding boxes
gsds = batch["gsd"]  # list of ground sampling distances for the images
labels = batch["label"]  # class label of the instance (e.g. "airport")
subsets = batch["subset"]  # subset label of the instance (e.g. "train")
assert len(np.unique(labels)) == 1 and len(np.unique(subsets)) == 1
class_name = instance_parser.metadata.class_names[labels[0]]
subset_name = instance_parser.metadata.subset_types[subsets[0]]
print(f"\tclass={class_name}, subset={subset_name}")
image_count = len(images)
if image_count > max_display_count:
    print(f"\tnum images: {image_count} (will display first {max_display_count})")
else:
    print(f"\tnum images: {image_count}")
display_count = min(max_display_count, image_count)

fig, axes = plt.subplots(nrows=1, ncols=display_count, squeeze=False, figsize=(5 * display_count, 5))
for img_idx in range(display_count):
    image = torchvision.transforms.functional.to_pil_image(images[img_idx])
    drawer = PIL.ImageDraw.ImageDraw(image)
    bbox = bboxes[img_idx]
    left, top = bbox[0], bbox[1]
    right, bottom = left + bbox[2], top + bbox[3]
    drawer.rectangle((left, top, right, bottom), width=10, outline="red")
    axes[0][img_idx].imshow(image, aspect="auto")
    axes[0][img_idx].title.set_text(f"GSD = {gsds[img_idx]:.2f}m")

In [ ]:
image_parser = ssl4rs.data.parsers.fmow.DeepLakeParser(
    dataset_path_or_object=data_path,
    parsing_strategy="images",  # each 'sample' will be an image for a an instance
)
image_parser.summary()
# we can use the default visualizer to display image-wise examples
image_parser.visualize()